<a href="https://colab.research.google.com/github/o-fugi/FURSPColexification/blob/main/code/Mixed_Effect_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# run this cell and then restart the runtime if you want to generate special fancy graphs

# !pip install matplotlib --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.2 MB 5.3 MB/s 
     |████████████████████████████████| 959 kB 48.7 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


In [2]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/ColabFiles/

Mounted at /content/drive/
/content/drive/MyDrive/ColabFiles


In [3]:
import pandas as pd
import numpy as np
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from functools import reduce
import random as r
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# import gpboost as gpb

In [4]:
!pip install setuptools -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 2.0 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
numba 0.56.2 requires setuptools<60, but you have setuptools 65.4.1 which is incompatible.


In [5]:
!pip install gpboost -U
import gpboost as gpb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.0 MB 2.1 MB/s 


In [6]:
# import factor  datasets

# https://saifmohammad.com/WebPages/nrc-vad.html
val_df = pd.read_csv('/content/drive/MyDrive/ColabFiles/NRC-VAD-Lexicon.txt', sep="\t", header=None)
val_df = val_df.rename(columns={0:'Word', 1:'val', 2:'aro', 3:'dom'})[['Word', 'val', 'aro', 'dom']]

conc_df = pd.read_csv('/content/drive/MyDrive/ColabFiles/brysbaert_concreteness.csv')
conc_df = conc_df.rename(columns={'Conc.M': 'conc'})[['Word', 'conc']]

freq_df = pd.read_csv('/content/drive/MyDrive/ColabFiles/COCA_freqs.csv', encoding='ISO-8859-1') # w1, coca_spok
freq_df = freq_df.rename(columns={'w1':'Word', 'coca_spok':'freq'})[['Word', 'freq']]

In [7]:
# import semantic shift dataset

sem_shift_df = pd.read_csv('/content/drive/MyDrive/ColabFiles/cleaned_dat_sem_shift.csv')
sem_shift_df['meaning1'] = sem_shift_df['meaning1_clean']
sem_shift_df['meaning2'] = sem_shift_df['meaning2_clean']
sem_shift_df = sem_shift_df[['meaning1', 'meaning2', 'realizations']]

sem_shift_df.at[697, 'meaning1'] = 'furuncul'
sem_shift_df.at[1521, 'meaning2'] = 'geometrid'

In [8]:
# get the dataset all figured out 
datSemShift = pd.read_csv('/content/drive/MyDrive/ColabFiles/datsemshift.csv')

# filter semantic shifts
datSemShift = datSemShift[(datSemShift['gendirection'] == '→')]
datSemShift = datSemShift[(datSemShift['type'] == ' Semantic evolution') | (datSemShift['type'] == ' Polysemy')]
datSemShift = datSemShift[(datSemShift['language1'] == datSemShift['language2'])]
datSemShift = datSemShift[(datSemShift['lexeme1'] == datSemShift['lexeme2'])]
datSemShift = datSemShift[~datSemShift['meaning1'].str.contains('<')]
datSemShift = datSemShift[~datSemShift['meaning2'].str.contains('<')]
datSemShift = datSemShift[(datSemShift['status']!='Suspended') & (datSemShift['status']!='Rejected')]
datSemShift = datSemShift[['meaning1', 'meaning2', 'language1']]
datSemShift = datSemShift.reset_index(drop=True)

In [9]:
dumb_british_spellings = {"vapour":"vapor", "honour":"honor", "organisation":"organization", "harbour":"harbor", "odour":"odor", "centre":"center", "analyse":"analyze", "theatre":"theater", "colour":"color", "rumour":"rumor", "behaviour":"behavior", "armour":"armor", "grey":"gray", 
"mould":"mold", "neighbour":"neighbor", "axe":"ax", "moustache":"mustache", "plough":"plow", "mandarine":"mandarin"}
words_to_replace = {"adj":"", "gipsy":"", "albumen":"", "campanula":"", "boletus edulis":"penny bun fungus", "ursus":"", "swearword":"swear word", 'adj.':"", "coleus":"", 'n.':"", 'ОК':"ok", "typha":"cattail", "pacifica":"peaceful", "mustella":"", "smail":"", "one’s":"", "spurflower":"perennial plant", "sabre":"","equus":"", "etc.":"", "ciconia":"",  "aër":"", "panthera":"panther", "еrinaceus":"", "e.g.":"", "centaurea":"thistle", "moschiferus":"", "apterus":"", "pyrrhocoris":"", "smn.":"", "pritchardia":"", "100":"one hundred", "sabrefish":"sabre carp", "putorius":"", "adv.":"", "petromyzontidae":"", "botaurus":"", "standart":"standard", "leccinum":"", "sg.":"", "gemini":"Gemini", "tabanidae":"", "anagallis":"", "decorticate":"stiff", "albugo":"", "frangula":"", "sciurus":"", "scrofa":"", "relig.":"", "headstream":"head stream", "solanum":"", "anguilla":"", "anat.":"", "nectarinia":"", "ipomoea":"", "repaire":"repair", "vaccinium":"", "smth":"", "smth.":"", "bubo":"", "deflorate":"remove flowers", "tr.":"", "traveller":"traveler", "bubalis":"", "marmorata":"", "furuncul":"", "caballus":"", "microchiroptera":"", "urtica":"", "plumbum":"", "biol.":"", "intr.":"", "bubalus":"", "columba":"", "cucurbita":"", "goldcrest":"small bird", "melongena":"", "picea":"", "arvensis":"", "moschus":"", "psidium":"", "radiointerference":"radio interference", "owre":"", "ricinus":"", "capricorn":"goat zodiac sign", "mustela":"", "pandion":"", "adj.of":"", "nomadize":"become nomadic", "smb.":"", "kneepit":"knee pit", "num.":"", "pl.":"", "extortioner":"extortion doer", "enculturate":"assimilate", "asquint":"squint", "uliginosum":"", "heteroptera":"", "ок":"ok", "abies":"fir", "stratiotes":"", "fiddlestick":"violin bow", "scabrum":"", "grus":"bird", "acarina":"", "guajava":"", "bitterling":"freshwater fish", "lycopersicum":"", "lutra":"otter", "plectranthus":"", "macereed":"mace reed", "24":"twenty four", "acris":"", "rotundifolius":"common weed", "gutturalis":"", "oxyeleotris":"", "geometrid":"", "citrullus":"", "lepus":"", "motacilla":"", "crake":"bird", "haliaёtus":"", "glasswort":"herb", "quinsy":"throat abscess", "shoulderblade":"shoulder blade", "spearthrower":"spear thrower", "ridgepole":"ridge pole", "pimpleface":"pimple face", "tumpline":"backpack", "cushma":"clothing", "curassow":"tropical bird", "banisterium":"plant", "paca":"rodent", "netbag":"net bag", "muntjacs":"barking deer"}
#Replaces obscure words with recognizable words, replaces parts of speech with nothing, same with science words

def clean(s): # Here is the main method where we clean senses
  s = str(s)
  for i in ",:":
    s = s.replace(i, " or")
  s = s.replace("/", " or ")
  for i in "()\t\n<>?’":
    s = s.replace(i, "")
  for i in "-":
    s = s.replace(i, " ")
  should_be_upper = {}
  for w in s.split(" "):
    if w != "" and w[0].isupper():
      should_be_upper = {w.lower():w.capitalize()} # Keeps track of which words should be capitalized
  s = s.lower() # Converts to lowercase
  for i in dumb_british_spellings: # Replaces British spelling with American ones
    s = s.replace(i, dumb_british_spellings[i])
  for i in words_to_replace: # Replaces all other obscure words 
    s = s.replace(i, words_to_replace[i])
  for i in should_be_upper: # Uppercases words which should be uppercase
    s = s.replace(i, should_be_upper[i])
  if len(s) > 2 and s[:3] == " or": # If the beginning word is or, that means our string was of the form "[removed word]," and we should delete the or
    s = s[3:]
  if "vs." in s: # For cases like "land (vs. sea)"" we want to convert this to "land"
    ind = s.find("vs.")
    s = s[:ind]
  s = s.replace(".","")
  return s

df_raw = datSemShift
sem_shift_df = df_raw.copy()
sem_shift_df['meaning1'] = sem_shift_df["meaning1"].apply(clean)
sem_shift_df['meaning2'] = sem_shift_df["meaning2"].apply(clean)

In [10]:
# define how we map a sense to concreteness, valence, frequency values

skip_words = {'the', 'and', 'of', 'a', 'in', 'to', 'it', 'is', 'that', 'for', 'with', 'on', 'at', 'this', 'that', 'by', 'or', 'an', 'as', 'if', 'so', 'in'}

# take all words in the sense and and average/add values
# if conservative, only matches to full phrases separated by "or"
def senseToValue(s, df, col, average=True, conservative=False):
  s = s.lower()
  if conservative: 
    phrases = re.split(' or ', s)
  else:
    for i in "(),/":
      s = s.replace(i, "")
    phrases = re.split(' ', s)
  total_val = 0
  val_data = 0
  for p in phrases:
    if p in skip_words:
      continue
    try:
      val = df[df['Word']==p][col].values[0]
      if not np.isnan(val):
        total_val += val
        val_data += 1
    except:
      pass
  if val_data == 0:
    total_val = np.nan
  else:
    if average:
      total_val /= val_data
  return total_val

In [11]:
# get priors for semshift data

# dictionary for concreteness and embeddings
conc_dic = {} # This will be a dictionary that easily allows us to access the concreteness for all of our senses, saving time. 
freq_dic = {} 
val_dic = {}
val_rate_dic = {}

conservative=True

for i in range(len(sem_shift_df)): # Here we loop through each row of our dataframe, and if we can convert a sense s to an embedding then we set vec_dic[s] = embedding
  row = sem_shift_df.iloc[i]
  x = row["meaning1"]
  y = row["meaning2"]

  if x not in conc_dic:
    x_conc = senseToValue(x, conc_df, 'conc', conservative=conservative)
    if not np.isnan(x_conc):
      conc_dic[x] = x_conc

  if y not in conc_dic:
    y_conc = senseToValue(y, conc_df, 'conc', conservative=conservative)
    if not np.isnan(y_conc):
      conc_dic[y] = y_conc

  if x not in freq_dic:
    x_freq = senseToValue(x, freq_df, 'freq', average=True, conservative=conservative)
    if not np.isnan(x_freq):
      freq_dic[x] = x_freq

  if y not in freq_dic:
    y_freq = senseToValue(y, freq_df, 'freq', average=True, conservative=conservative)
    if not np.isnan(y_freq):
      freq_dic[y] = y_freq

  if x not in val_dic:
    x_val = senseToValue(x, val_df, 'val', conservative=conservative)
    if not np.isnan(x_val):
      val_dic[x] = abs(x_val - 0.5)

  if y not in val_dic:
    y_val = senseToValue(y, val_df, 'val', conservative=conservative)
    if not np.isnan(y_val):
      val_dic[y] = abs(y_val - 0.5)

In [ ]:
#@title export these as a dataframe, for use in other places

# conc_sense_df = pd.DataFrame.from_dict(conc_dic, orient='index').reset_index().rename(columns={'index':'Word', 0:'conc'})
# freq_sense_df = pd.DataFrame.from_dict(freq_dic, orient='index').reset_index().rename(columns={'index':'Word', 0:'freq'})
# val_sense_df = pd.DataFrame.from_dict(val_dic, orient='index').reset_index().rename(columns={'index':'Word', 0:'val'})

# df = reduce(lambda  left,right: pd.merge(left,right,left_on='meaning1',right_on='Word', how='left'), [sem_shift_df[['meaning1', 'meaning2']], conc_sense_df])
# df = reduce(lambda  left,right: pd.merge(left,right,left_on='meaning2',right_on='Word', how='left'), [df, conc_sense_df])
# df = reduce(lambda  left,right: pd.merge(left,right,left_on='meaning1',right_on='Word', how='left'), [df, freq_sense_df])
# df = reduce(lambda  left,right: pd.merge(left,right,left_on='meaning2',right_on='Word', how='left'), [df, freq_sense_df])
# df = reduce(lambda  left,right: pd.merge(left,right,left_on='meaning1',right_on='Word', how='left'), [df, val_sense_df])
# df = reduce(lambda  left,right: pd.merge(left,right,left_on='meaning2',right_on='Word', how='left'), [df, val_sense_df])

# df = df.drop(['Word_x', 'Word_y'], axis=1)

# df.to_csv('all_vars_df.csv', index=False)

In [12]:
# get group indices
lang_df = sem_shift_df.groupby('language1').count().reset_index()
lang_df = lang_df[lang_df['meaning1'] >=10]
lang_list = lang_df['language1'].values
sem_shift_df['language'] = sem_shift_df['language1'].map({y: x for x, y in dict(enumerate(lang_list)).items()})

In [34]:
# set up training and testing databases

# get a dataframe with all the concreteness values and drop NaN
conc_sense_df = pd.DataFrame.from_dict(conc_dic, orient='index').reset_index().rename(columns={'index':'Word', 0:'conc'})
freq_sense_df = pd.DataFrame.from_dict(freq_dic, orient='index').reset_index().rename(columns={'index':'Word', 0:'freq'})
val_sense_df = pd.DataFrame.from_dict(val_dic, orient='index').reset_index().rename(columns={'index':'Word', 0:'val'})
shift_val_df = reduce(lambda  left,right: pd.merge(left,right,left_on='meaning1',right_on='Word', how='left'), [sem_shift_df, conc_sense_df]).drop('Word', axis=1)
shift_val_df = reduce(lambda  left,right: pd.merge(left,right,left_on='meaning2',right_on='Word', how='left'), [shift_val_df, conc_sense_df]).drop('Word', axis=1)
shift_val_df = reduce(lambda  left,right: pd.merge(left,right,left_on='meaning1',right_on='Word', how='left'), [shift_val_df, freq_sense_df]).drop('Word', axis=1)
shift_val_df = reduce(lambda  left,right: pd.merge(left,right,left_on='meaning2',right_on='Word', how='left'), [shift_val_df, freq_sense_df]).drop('Word', axis=1)
shift_val_df = reduce(lambda  left,right: pd.merge(left,right,left_on='meaning1',right_on='Word', how='left'), [shift_val_df, val_sense_df]).drop('Word', axis=1)
shift_val_df = reduce(lambda  left,right: pd.merge(left,right,left_on='meaning2',right_on='Word', how='left'), [shift_val_df, val_sense_df]).drop('Word', axis=1)
shift_val_df = shift_val_df.dropna().reset_index(drop=True)

shift_val_df['conc_diff'] = shift_val_df['conc_y'] - shift_val_df['conc_x']
shift_val_df['freq_diff'] = shift_val_df['freq_y'] - shift_val_df['freq_x']
shift_val_df['val_diff'] = shift_val_df['val_y'] - shift_val_df['val_x']

train_df = shift_val_df[['conc_diff', 'freq_diff', 'val_diff', 'language1']].copy() 

# create a train dataframe where half of the directions are switched 
indices = list(range(len(shift_val_df)))
r.shuffle(indices)
switch_indices = indices[:int(len(shift_val_df)/2)]

train_df['accurate'] = pd.Series(dtype='int')
train_df.loc[switch_indices, 'accurate'] = 0
train_df.loc[indices[int(len(train_df)/2):], 'accurate'] = 1

train_df.loc[switch_indices, 'conc_diff'] = -shift_val_df.loc[switch_indices, 'conc_diff']
train_df.loc[switch_indices, 'freq_diff'] = -shift_val_df.loc[switch_indices, 'freq_diff']
train_df.loc[switch_indices, 'val_diff'] = -shift_val_df.loc[switch_indices, 'val_diff']

In [39]:
# do logistic regression as a baseline

X_train, X_test, y_train, y_test = train_test_split(train_df[['conc_diff', 'freq_diff', 'val_diff', 'language1']], train_df['accurate'].values, test_size=0.2, random_state=0)

model = LogisticRegression(solver='liblinear', random_state=0)
model.fit(X_train[['conc_diff']].values, y_train)
print("Concreteness:", model.score(X_test[['conc_diff']].values, y_test))

model = LogisticRegression(solver='liblinear', random_state=0)
model.fit(X_train[['freq_diff']].values, y_train)
print("Frequency:", model.score(X_test[['freq_diff']].values, y_test))

model = LogisticRegression(solver='liblinear', random_state=0)
model.fit(X_train[['val_diff']].values, y_train)
print("Valence:", model.score(X_test[['val_diff']].values, y_test))

Concreteness: 0.7420718816067653
Frequency: 0.4630021141649049
Valence: 0.6680761099365751


In [61]:
# GLMM

# this is weird beacuse it's not really solving a classification problem? it's a linear fit problem. 

X_train['intercept'] = 1
X_test['intercept'] = 1

gp_model_conc = gpb.GPModel(group_data=X_train['language1'], likelihood='bernoulli_logit')  #either "gaussian" (=regression), "bernoulli_probit", "bernoulli_logit", (=classification)"poisson", or "gamma"
gp_model_conc.fit(y=y_train, X=X_train[['conc_diff', 'intercept']], params={'std_dev': True})
print("Concreteness")
print(gp_model_conc.summary())

gp_model_freq = gpb.GPModel(group_data=X_train['language1'], likelihood='bernoulli_logit')  #either "gaussian" (=regression), "bernoulli_probit", "bernoulli_logit", (=classification)"poisson", or "gamma"
gp_model_freq.fit(y=y_train, X=X_train[['freq_diff', 'intercept']], params={'std_dev': True})
print("Freq")
print(gp_model_freq.summary())

gp_model_val = gpb.GPModel(group_data=X_train['language1'], likelihood='bernoulli_logit')  #either "gaussian" (=regression), "bernoulli_probit", "bernoulli_logit", (=classification)"poisson", or "gamma"
gp_model_val.fit(y=y_train, X=X_train[['val_diff', 'intercept']], params={'std_dev': True})
print("Val")
print(gp_model_val.summary())

Concreteness
Model summary:
 Log-lik     AIC     BIC
 -978.33 1962.66 1979.29
Nb. observations: 1889
Nb. groups: 132 (language1)
---------------------------------------------------
Covariance parameters (random effects):
           Param.
language1  0.0123
---------------------------------------------------
Linear regression coefficients (fixed effects):
           Param.  Std. dev.  z value  P(>|z|)
conc_diff -1.3027     0.0664 -19.6239   0.0000
intercept  0.0545     0.0571   0.9557   0.3392
Freq
Model summary:
 Log-lik     AIC     BIC
-1298.36 2602.73 2619.36
Nb. observations: 1889
Nb. groups: 132 (language1)
---------------------------------------------------
Covariance parameters (random effects):
           Param.
language1  0.0034
---------------------------------------------------
Linear regression coefficients (fixed effects):
           Param.  Std. dev.  z value  P(>|z|)
freq_diff  0.0000     0.0000   4.4185   0.0000
intercept -0.0049     0.0469  -0.1051   0.9163
Val
Model su

In [63]:
# get accuracies for test 

pred = gp_model_conc.predict(X_pred=X_test[['conc_diff', 'intercept']], group_data_pred=X_test['language1'],
                        predict_var=True, predict_response=False)
accurate = [y_test[i] == (1 if pred['mu'][i]> 0 else 0) for i in range(len(y_test))]
print('Conc accuracy', accurate.count(True)/len(accurate))

pred = gp_model_freq.predict(X_pred=X_test[['freq_diff', 'intercept']], group_data_pred=X_test['language1'],
                        predict_var=True, predict_response=False)
accurate = [y_test[i] == (1 if pred['mu'][i]> 0 else 0) for i in range(len(y_test))]
print('Freq accuracy', accurate.count(True)/len(accurate))

pred = gp_model_val.predict(X_pred=X_test[['val_diff', 'intercept']], group_data_pred=X_test['language1'],
                        predict_var=True, predict_response=False)
accurate = [y_test[i] == (1 if pred['mu'][i]> 0 else 0) for i in range(len(y_test))]
print('Val accuracy', accurate.count(True)/len(accurate))

Conc accuracy 0.7441860465116279
Freq accuracy 0.4714587737843552
Val accuracy 0.6638477801268499


In [19]:
# get coefficients for each language

all_training_data_random_effects = gp_model_conc.predict_training_data_random_effects()
first_occurences = [np.where(X_train['language1']==i)[0][0] for i in np.unique(X_train['language1'])]
training_data_random_effects = all_training_data_random_effects.iloc[first_occurences].copy()
training_data_random_effects['language'] = np.unique(X_train['language1'])

In [20]:
training_data_random_effects

,language1,language
462,0.024511,Abaza
148,0.016003,Adyghe
765,0.014948,Aghul
76,-0.067963,Akkadian
139,0.055759,Amharic
...,...,...
503,0.054471,Waray
1089,-0.025446,Welsh
37,-0.010031,Yakut
56,0.005253,Yaqui (Hiaki)
